<a href="https://colab.research.google.com/github/mralamdari/Machine_Learning_Projects/blob/main/Digit_Classifier_with_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Approch

In [ ]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
mnist = sklearn.datasets.fetch_openml('mnist_784', version=1)

In [ ]:
x, y = mnist.data, mnist.target

In [ ]:
X.shape

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, random_state=77, test_size=0.2)

In [ ]:
def standardization(x):
    return (x - np.mean(x, axis=0)) / np.std(x, axis=0)

x_train_temp = standardization(x_train)
x_test_temp = standardization(x_test)
x_train_std, x_test_std = np.nan_to_num(x_train_temp), np.nan_to_num(x_test_temp)


clf = sklearn.ensemble.RandomForestClassifier()
clf.fit(x_train, y_train)
sgd_pred = clf.predict(x_test)
sklearn.metrics.accuracy_score(y_test, sgd_pred)

# Deep Learning Approch

In [ ]:
import torch
import torchvision

In [ ]:
input_size=784     # images 784*784
out_size=10        # 10 numbers
epochs=10          # Iterations   
batch_size=100     
learning_rate=0.001 

In [ ]:
mean_gray = 0.1307
std_grey  = 0.3081

transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                             torchvision.transforms.Normalize((mean_gray, ), (std_grey, ))])

In [ ]:
train_ds = torchvision.datasets.MNIST(root='/data',
                                      train = True,
                                      transform = torchvision.transforms.ToTensor(),
                                      download=True)

In [ ]:
test_ds = torchvision.datasets.MNIST(root='/data',
                                      train=False,
                                      transform = torchvision.transforms.ToTensor(),
                                      download=True)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_ds,
                                           batch_size=batch_size,
                                           shuffle=True)

In [ ]:
test_loader = torch.utils.data.DataLoader(test_ds, 
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
    self.batchnorm1 = torch.nn.BatchNorm2d(8)
    self.relu = torch.nn.ReLU()
    self.maxpool = torch.nn.MaxPool2d(kernel_size=2)
    self.conv2 = torch.nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5, stride=1, padding=2)
    self.batchnorm2 = torch.nn.BatchNorm2d(32)
    self.fc1 = torch.nn.Linear(32*7*7, 600)
    self.dropout = torch.nn.Dropout(p=0.5)
    self.fc2 = torch.nn.Linear(600, 100)

  def forward(self, x):
    out = self.conv1(x)
    out = self.batchnorm1(out)
    out = self.relu(out)  
    out = self.maxpool(out)
    out = self.conv2(out)
    out = self.batchnorm2(out)
    out = self.relu(out)
    out = self.maxpool(out)

    out = out.view(-1, 1568)

    out = self.fc1(out)
    out = self.dropout(out)
    out = self.fc2(out)
    return out

In [ ]:
model = Net()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
num_epochs=20
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []

for epoch in range(num_epochs):
  correct = 0
  iterations = 0
  iter_loss = 0.0

  model.train()
  for i, (inputs, labels) in enumerate(train_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = loss_fn(outputs, labels)
    iter_loss += loss.item()
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()

    _, pred = torch.max(outputs, 1)
    correct += (pred == labels).sum().item()
    iterations += 1
  
  train_loss.append(iter_loss/iterations)
  train_accuracy.append(100 * correct/ len(train_ds))
  
  t_loss = 0.0
  correct = 0
  iterations = 0
  model.eval()
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    loss = loss_fn(outputs, labels)
    t_loss += loss.item()
    _, pred = torch.max(outputs, 1) #(100, 10)  ==> 10 is in index 1
    correct += (pred == labels).sum().item()
    iterations += 1

    test_loss.append(t_loss/iterations)
    test_accuracy.append(100*correct/len(test_ds))


  print(f'epoch: {epoch}, Training Loss: {train_loss[-1]:.3f}, Training Accuracy: %{train_accuracy[-1]:.3f}, Test Loss: {test_loss[-1]:.3f}, Test Accuracy:% {test_accuracy[-1]:.3f}')

epoch: 0, Training Loss: 0.119, Training Accuracy: %97.508, Test Loss: 0.131, Test Accuracy:% 97.760
epoch: 1, Training Loss: 0.110, Training Accuracy: %97.672, Test Loss: 0.106, Test Accuracy:% 97.330
epoch: 2, Training Loss: 0.109, Training Accuracy: %97.673, Test Loss: 0.086, Test Accuracy:% 98.390
epoch: 3, Training Loss: 0.115, Training Accuracy: %97.658, Test Loss: 0.145, Test Accuracy:% 97.490
epoch: 4, Training Loss: 0.121, Training Accuracy: %97.523, Test Loss: 0.110, Test Accuracy:% 97.910
epoch: 5, Training Loss: 0.102, Training Accuracy: %97.762, Test Loss: 0.114, Test Accuracy:% 97.890
epoch: 6, Training Loss: 0.112, Training Accuracy: %97.652, Test Loss: 0.096, Test Accuracy:% 97.720
epoch: 7, Training Loss: 0.115, Training Accuracy: %97.557, Test Loss: 0.095, Test Accuracy:% 97.620
epoch: 8, Training Loss: 0.121, Training Accuracy: %97.507, Test Loss: 0.109, Test Accuracy:% 98.030
epoch: 9, Training Loss: 0.117, Training Accuracy: %97.625, Test Loss: 0.144, Test Accuracy

In [ ]:
fig, axs = plt.subplots(6, 6, figsize = (20, 20))
plt.gray()

# loop through subplots and add centroid images
for i, ax in enumerate(axs.flat):
    
    # determine inferred label using cluster_labels dictionary
    for key, value in cluster_labels.items():
        if i in value:
            ax.set_title('Inferred Label: {}'.format(key))
    
    # add image to subplot
    ax.matshow(images[i])
    ax.axis('off')
    
# display the figure
fig.show()